## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
#Drop any missing data

attrition_df.dropna(inplace=True)

In [5]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [6]:
# Create a list of at least 10 column names to use as X data

columns = ['YearsSinceLastPromotion', 'YearsInCurrentRole', 'TrainingTimesLastYear', 'StockOptionLevel', 'EnvironmentSatisfaction', 'PercentSalaryHike', 'OverTime', 'JobRole', 'JobInvolvement','Age']

# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes




YearsSinceLastPromotion     int64
YearsInCurrentRole          int64
TrainingTimesLastYear       int64
StockOptionLevel            int64
EnvironmentSatisfaction     int64
PercentSalaryHike           int64
OverTime                   object
JobRole                    object
JobInvolvement              int64
Age                         int64
dtype: object

In [7]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform the 'OverTime' column of X_train
X_train['OverTime'] = le.fit_transform(X_train['OverTime'].values.reshape(-1,1))

# Transform the 'OverTime' column of X_test using the same encoder
X_test['OverTime'] = le.transform(X_test['OverTime'].values.reshape(-1,1))

C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [9]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# Create encoder
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(X_train[['JobRole']])

# Apply the encoder to the training and testing data
X_train_encoded = encoder.transform(X_train[['JobRole']])
X_train_encoded
X_test_encoded = encoder.transform(X_test[['JobRole']])

# Convert the encoded features into DataFrames
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(['JobRole']))
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(['JobRole']))

# Drop the original 'JobRole' column from the original df
X_train = X_train.drop(['JobRole'], axis=1)
X_test = X_test.drop(['JobRole'], axis=1)

# Reset indices for X_train data
X_train = X_train.reset_index(drop=True)
X_train_encoded_df = X_train_encoded_df.reset_index(drop=True)

# Concatenate the original DataFrames with the encoded DataFrames
X_train = pd.concat([X_train, X_train_encoded_df], axis=1)

# Reset indices for X_test data
X_test = X_test.reset_index(drop=True)
X_test_encoded_df = X_test_encoded_df.reset_index(drop=True)

# Concatenate the original DataFrames with the encoded DataFrames
X_test = pd.concat([X_test, X_test_encoded_df], axis=1)

# View the first few rows of X_train
X_train.head()

,YearsSinceLastPromotion,YearsInCurrentRole,TrainingTimesLastYear,StockOptionLevel,EnvironmentSatisfaction,PercentSalaryHike,OverTime,JobInvolvement,Age,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,0,1,3,3,3,14,0,2,24,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,2,0,2,15,0,3,18,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,2,2,1,4,14,0,3,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,4,3,0,4,21,0,3,39,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,8,2,1,2,11,0,3,31,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
# Split y_df
y_train_attrition = y_train['Attrition']
y_train_department = y_train['Department']
y_test_attrition = y_test['Attrition']
y_test_department = y_test['Department']

y_train_attrition

1097     No
727      No
254      No
1175     No
1341     No
       ... 
1130     No
1294     No
860     Yes
1459     No
1126     No
Name: Attrition, Length: 1176, dtype: object

In [12]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train_attrition.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_attrition = encoder.transform(y_train_attrition.values.reshape(-1, 1))
y_test_encoded_attrition = encoder.transform(y_test_attrition.values.reshape(-1, 1))


In [13]:
# Create a OneHotEncoder for the Department column
encoder_department = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder_department.fit(y_train_department.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = encoder_department.transform(y_train_department.values.reshape(-1, 1))
y_test_department_encoded = encoder_department.transform(y_test_department.values.reshape(-1, 1))

## Create, Compile, and Train the Model

In [14]:
# Find the number of columns in the X training data
num_cols = X_train.shape[1]
print(f"Number of columns: {num_cols}")

# Create the input layer
input_layer = layers.Input(shape=(num_cols,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

Number of columns: 18


In [15]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
hidden_dept = layers.Dense(32, activation='relu')(shared_layer2)


# Create the output layer
output_dept = layers.Dense(3, activation='softmax', name="dept_output")(hidden_dept)


In [16]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
hidden_attrition = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
output_attrition = layers.Dense(2, activation='sigmoid', name='attrition_output')(hidden_attrition)

In [17]:
# Create the model
model = Model(inputs=input_layer, outputs=[output_dept, output_attrition])


# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'accuracy'])


# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      1,216 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      1,056 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      1,056 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │         99 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,573 (21.77 KB)

 Trainable params: 5,573 (21.77 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model

history = model.fit(
    X_train_scaled, [y_train_department_encoded, y_train_encoded_attrition],
    epochs=100,
    batch_size=32,
)

Epoch 1/100


37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 914us/step - attrition_output_accuracy: 0.8192 - dept_output_accuracy: 0.6036 - loss: 1.2002
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - attrition_output_accuracy: 0.8452 - dept_output_accuracy: 0.6696 - loss: 0.7897
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - attrition_output_accuracy: 0.8367 - dept_output_accuracy: 0.9448 - loss: 0.5957
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step - attrition_output_accuracy: 0.8425 - dept_output_accuracy: 0.9775 - loss: 0.4689
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - attrition_output_accuracy: 0.8398 - dept_output_accuracy: 0.9574 - loss: 0.4169
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - attrition_output_accuracy: 0.8408 - dept_output_accuracy: 0.9667 - loss: 0.4081
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - attrition_output_accuracy: 0.8499 - dept_output_accuracy: 0.9695 - loss: 0.3810
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 695us/step - attrition_ou

In [19]:
# Evaluate the model with the testing data
evaluate_score = model.evaluate(X_test_scaled, [y_test_department_encoded, y_test_encoded_attrition])

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.7801 - dept_output_accuracy: 0.9522 - loss: 1.5059  


In [20]:
# Print the accuracy for both department and attrition
evaluate_score

[1.4339685440063477, 0.795918345451355, 0.9489796161651611]

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. I would say that accuracy was the best metric to use. It was identified that there was overfittig because of that metric.
2. Since there was 3 classes in the department column, I decided to use softmax since it wasn't binary. Similarly, I used sigmoid for the attrition column because it had only 2 classes making it binary.
3. The model can be refined in many ways. I can go back and preprocess better to accomadate for overfitting and I could also try another model structure.